![title](./logo_nao_digital.png)

# **Tienes un mensaje**

## Evidencias ciclo ID1

*Instrucciones:*


Atiende las siguientes indicaciones para completar las evidencias de este primer ciclo de investigación y desarrollo de esta experiencia de aprendizaje.

1. Lee detenidamente el reto Tienes un mensaje.

2. Desarrolla las evidencias solicitadas para este ciclo ID:
* Crea un script en Python para leer una lista de correos de un CSV y enviar un saludo en un correo electrónico.
* Crea un script en Bash para invocar cada lunes este correo.
* Toma como evidencia el enlace al código en GitHub.
* Envía y recibe un correo electrónico y toma captura de pantalla.

3. Sube las evidencias a tu espacio en Notion.

## Etapa 1: instalar e importar librerías de trabajo

Primero instalamos e importamos las librerías de trabajo

In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import smtplib
import openpyxl
import csv

# Import the email modules we'll need
from email.message import EmailMessage

import warnings
warnings.filterwarnings('ignore')

## Etapa 2: generar una lista de nombres, correos y mensajes CSV

In [3]:
# Primero generamos una lista de nombres de alguna base de datos de la red
# Se tratan de 500 empleados, elegiremos 250 hombre y 250 mujeres

# Censo nombre de Brasil en formato json
pd.set_option('display.max_rows', None)

nombres_m = pd.read_json("https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking/?sexo=M&qtd=250")
nombres_f = pd.read_json("https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking/?sexo=F&qtd=250")

In [4]:
print("Cantidad de nombres: " + str(len(nombres_m) + len(nombres_f)))

Cantidad de nombres: 500


In [5]:
# La información está almacenada como lista, ahora la pasamos a DataFrame
nombres = [nombres_m, nombres_f]
nombres

[           nome  regiao     freq  rank sexo
 0          JOSE       0  5732508     1    M
 1          JOAO       0  2971935     2    M
 2       ANTONIO       0  2567494     3    M
 3     FRANCISCO       0  1765197     4    M
 4        CARLOS       0  1483121     5    M
 5         PAULO       0  1417907     6    M
 6         PEDRO       0  1213557     7    M
 7         LUCAS       0  1116818     8    M
 8          LUIZ       0  1102927     9    M
 9        MARCOS       0  1101126    10    M
 10         LUIS       0   931530    11    M
 11      GABRIEL       0   922744    12    M
 12       RAFAEL       0   814709    13    M
 13       DANIEL       0   706527    14    M
 14      MARCELO       0   690098    15    M
 15        BRUNO       0   663271    16    M
 16      EDUARDO       0   628539    17    M
 17       FELIPE       0   615924    18    M
 18     RAIMUNDO       0   611174    19    M
 19      RODRIGO       0   598825    20    M
 20       MANOEL       0   590924    21    M
 21       

In [6]:
# Vamos a dejar solamente la columna nome (de una vez renombrándola por nombre)
nombres = pd.concat(nombres)['nome'].to_frame()
nombres.columns = ['nombre']
nombres.sample(5)

,nombre
42,MATHEUS
56,DOUGLAS
114,ELZA
14,JESSICA
234,KEILA


In [7]:
# Ahora generamos los servidores de correos y los asignamos de manera aleatoria
dominio = ['@emailbrokenia.com']
nombres['dominio'] = np.random.choice(dominio, len(nombres))
nombres.sample(5)

,nombre,dominio
162,ISMAEL,@emailbrokenia.com
121,ADEMIR,@emailbrokenia.com
20,MANOEL,@emailbrokenia.com
114,ARTHUR,@emailbrokenia.com
158,INGRID,@emailbrokenia.com


In [8]:
# Asignamos e-mails con nombre + dominio, teniendo cuidado con las mayúsculas
nombres['email'] = nombres.nombre.str.cat(nombres.dominio).str.lower()
nombres.sample(5)

,nombre,dominio,email
179,IAGO,@emailbrokenia.com,iago@emailbrokenia.com
18,RAIMUNDO,@emailbrokenia.com,raimundo@emailbrokenia.com
176,CELIO,@emailbrokenia.com,celio@emailbrokenia.com
68,LUIZA,@emailbrokenia.com,luiza@emailbrokenia.com
55,DANIELE,@emailbrokenia.com,daniele@emailbrokenia.com


In [9]:
# Asignamos la columna mensaje, el usuario deberá ingresar el mensaje deseado 
mensaje = input('Ingrese mensaje para inicio de la semana: ')
nombres['mensaje'] = mensaje
nombres.sample(5)

,nombre,dominio,email,mensaje
226,VILMAR,@emailbrokenia.com,vilmar@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
234,WELITON,@emailbrokenia.com,weliton@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
74,ALAN,@emailbrokenia.com,alan@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
81,LUAN,@emailbrokenia.com,luan@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
15,BRUNO,@emailbrokenia.com,bruno@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."


In [10]:
# Eliminamos la columna dominio
nombres = nombres.drop(columns=['dominio'])
nombres.sample(5)

,nombre,email,mensaje
131,ANGELICA,angelica@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
180,LIDIA,lidia@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
214,KAUA,kaua@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
155,CECILIA,cecilia@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."
24,FABIO,fabio@emailbrokenia.com,"hoy comienza la mejor semana de tu vida, ¡feli..."


In [11]:
# Ahora tenemos la base de datos generada, podemos guardarla como CSV
nombres.to_csv('BD_BrokenIA.csv', index = False)

## Etapa 3: leer la lista de nombres, correos y mensajes del CSV y enviar email

In [15]:
# Configuración de la conexión SMTP
smtp_server = 'smtp.gmail.com'
smtp_port = 587
smtp_user = 'nataliahoyos@gmail.com' 
smtp_password = 'iqpb razm awxa taqo'

# Dirección de correo de destino fija
correo_destino = 'nataliahoyos@gmail.com'

# Leer el archivo CSV
with open('BD_BrokenIA.csv', 'r', newline='') as csvfile:
    base_de_datos = csv.reader(csvfile)
    # Salta la primera fila para ignorar los encabezados 
    next(base_de_datos)

    # Contador para enviar a las primeras 10 personas
    contador = 0

    for row in base_de_datos:
        nombre = row[0]
        mensaje = row[2]

        # Crear unmensaje de correo electrónico
        email_message = EmailMessage()
        email_message.set_content(f'Hola, {nombre}: {mensaje}')

        email_message['From'] = smtp_user
        email_message['To'] = correo_destino
        email_message['Subject'] = 'Mensaje semanal'

        # Conectar y autenticar con el servidor SMTP
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Habilitar TLS si es necesario
            server.login(smtp_user, smtp_password)
            server.send_message(email_message)

        print(f'Correo enviado a {correo_destino} para {nombre}')
        
        # Incrementar el contador
        contador += 1

        # Salir del bucle después de enviar a las primeras 10 personas
        if contador >= 10:
            break


Correo enviado a nataliahoyos@gmail.com para JOSE
Correo enviado a nataliahoyos@gmail.com para JOAO
Correo enviado a nataliahoyos@gmail.com para ANTONIO
Correo enviado a nataliahoyos@gmail.com para FRANCISCO
Correo enviado a nataliahoyos@gmail.com para CARLOS
Correo enviado a nataliahoyos@gmail.com para PAULO
Correo enviado a nataliahoyos@gmail.com para PEDRO
Correo enviado a nataliahoyos@gmail.com para LUCAS
Correo enviado a nataliahoyos@gmail.com para LUIZ
Correo enviado a nataliahoyos@gmail.com para MARCOS


## Etapa 4: Acerca del funcionamiento del código principal: método de envío de correos

El código anterior tiene como objetivo enviar correos electrónicos personalizados a partir de un archivo CSV con los siguientes componentes:

1. **Configuración de la conexión SMTP:** Establece la configuración para la conexión al servidor SMTP de Gmail. Incluye el servidor SMTP (`smtp_server`), el puerto SMTP (`smtp_port`), la dirección de correo electrónico del remitente (`smtp_user`), y la contraseña de la aplicación generada para autenticación (`smtp_password`).

2. **Dirección de correo de destino fija:** Define la dirección de correo electrónico de destino, que en este caso es constante y se establece en `correo_destino` para efectos de realizar pruebas y asegurar la funcionalidad del código.

3. **Lectura del archivo CSV:** Abre y lee el archivo CSV denominado "BD_BrokenIA.csv" utilizando la biblioteca `csv`. El bucle `for` se utiliza para iterar a través de las filas del archivo CSV.

4. **Contador para enviar a las primeras 10 personas:** Se utiliza un contador (`contador`) para rastrear cuántos correos se han enviado. La idea es enviar correos personalizados a las primeras 10 personas en la base de datos y luego salir del bucle para probar el código

5. **Creación de un objeto de mensaje de correo electrónico:** Para cada persona en la base de datos, se crea un objeto `email_message` de la clase `EmailMessage` para representar el correo electrónico que se va a enviar. El cuerpo del correo electrónico se personaliza utilizando el nombre de la persona (`nombre`) y el mensaje de la fila actual del archivo CSV (`mensaje`).

6. **Configuración del correo electrónico:** Se establece el remitente (`From`), el destinatario (`To`), el asunto (`Subject`) y el contenido del mensaje del objeto `email_message`.

7. **Conexión y autenticación SMTP:** Se inicia una conexión al servidor SMTP de Gmail y se habilita el uso de TLS para seguridad. Luego, se autentica con la dirección de correo electrónico del remitente (`smtp_user`) y la contraseña de la aplicación generada (`smtp_password`).

8. **Envío del correo electrónico:** El correo electrónico se envía al destinatario (`correo_destino`) utilizando el objeto `email_message` configurado previamente.

9. **Impresión de un mensaje de confirmación:** Se imprime un mensaje para confirmar que el correo se ha enviado con éxito.

10. **Incremento del contador y salida del bucle:** El contador se incrementa en cada iteración. Cuando el contador alcanza 10, se utiliza la instrucción `break` para salir del bucle, lo que asegura que solo se envíen correos a las primeras 10 personas.

Este código permite automatizar el envío de correos electrónicos personalizados.